In [ ]:
import json
import pandas as pd
import numpy as np
import logging
from sqlalchemy import create_engine

# Setup logging
logging.basicConfig(filename='data_processing.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

def process_airbnb_data(file_path, save_path):
    # Load the JSON data
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    # Function to extract nested information
    def extract_nested_info(record):
        host_info = {
            'host_id': record.get('host', {}).get('host_id'),
            'host_name': record.get('host', {}).get('host_name'),
            'host_location': record.get('host', {}).get('host_location'),
            'host_is_superhost': record.get('host', {}).get('host_is_superhost'),
            'host_total_listings_count': record.get('host', {}).get('host_total_listings_count')
        }
        
        address_info = {
            'street': record.get('address', {}).get('street'),
            'suburb': record.get('address', {}).get('suburb'),
            'city': record.get('address', {}).get('government_area'),
            'country': record.get('address', {}).get('country'),
            'latitude': record.get('address', {}).get('location', {}).get('coordinates', [None, None])[1],
            'longitude': record.get('address', {}).get('location', {}).get('coordinates', [None, None])[0]
        }
        
        review_scores = record.get('review_scores', {})
        
        availability_info = {
            'availability_30': record.get('availability', {}).get('availability_30', 0),
            'availability_60': record.get('availability', {}).get('availability_60', 0),
            'availability_90': record.get('availability', {}).get('availability_90', 0),
            'availability_365': record.get('availability', {}).get('availability_365', 0)
        }
        
        processed_record = {
            'listing_id': record.get('_id'),
            'listing_name': record.get('name'),
            'property_type': record.get('property_type'),
            'room_type': record.get('room_type'),
            'price': record.get('price'),
            'accommodates': record.get('accommodates'),
            'bedrooms': record.get('bedrooms'),
            'bathrooms': record.get('bathrooms'),

            **host_info,
            **address_info,
            **availability_info,

            'review_score_rating': review_scores.get('review_scores_rating'),
            'review_score_accuracy': review_scores.get('review_scores_accuracy'),
            'review_score_cleanliness': review_scores.get('review_scores_cleanliness'),
            'review_score_location': review_scores.get('review_scores_location'),
            'review_score_value': review_scores.get('review_scores_value'),

            'minimum_nights': record.get('minimum_nights'),
            'number_of_reviews': record.get('number_of_reviews'),
            'cancellation_policy': record.get('cancellation_policy')
        }
        
        return processed_record

    # Process all records
    processed_data = [extract_nested_info(record) for record in data]
    
    # Convert to DataFrame
    df = pd.DataFrame(processed_data)
    
    # Feature Engineering
    def feature_engineering(df):
        logging.info("Starting feature engineering...")
        
        df.replace('-', '', regex=True, inplace=True)

        numeric_columns = ['price', 'accommodates', 'bedrooms', 
                           'bathrooms', 'review_score_rating',
                           'review_score_accuracy',
                           'review_score_cleanliness',
                           'availability_30',
                           'availability_60',
                           'availability_90',
                           'availability_365']
        
        for col in numeric_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col] = df[col].apply(lambda x: x if x >= 0 else np.nan)
            df[col].fillna(df[col].median(), inplace=True)

        df['is_superhost'] = df['host_is_superhost'].map({True: 1, False: 0})
        
        categorical_columns = ['property_type', 'room_type']
        
        df = pd.get_dummies(df, columns=categorical_columns)

        logging.info("Feature engineering completed.")
        
        return df

    # Apply feature engineering
    processed_df = feature_engineering(df)
    
    # Availability categorization
    def categorize_availability(availability):
        if availability == 0:
            return 'Not Available'
        elif availability < 30:
            return 'Low Availability'
        elif availability < 90:
            return 'Medium Availability'
        else:
            return 'High Availability'
    
    processed_df['availability_category'] = processed_df['availability_365'].apply(categorize_availability)
    
    # Save processed data
    processed_df.to_csv(save_path, index=False)
    logging.info(f'Data saved to CSV at {save_path} successfully.')
    
    return processed_df

def save_to_postgres(df, table_name, db_config):
    # Create a connection to PostgreSQL
    try:
        engine = create_engine(
            f"postgresql+psycopg2://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['dbname']}"
        )
        logging.info("Connected to PostgreSQL database.")

        # Save DataFrame to PostgreSQL
        df.to_sql(table_name, engine, if_exists='replace', index=False)
        logging.info(f"Data saved to PostgreSQL table '{table_name}' successfully.")
    except Exception as e:
        logging.error(f"Failed to save data to PostgreSQL: {str(e)}")

# Main execution
if __name__ == "__main__":
    file_path = r"C:\Users\User\Downloads\sample_airbnb (1).json"
    save_path = r"C:\Users\User\Desktop\jupiter_files\New1_processed_airbnb_data.csv"
    
    db_config = {
        'host': 'localhost',
        'port': '5432',
        'dbname': 'Airbnb',
        'user': 'postgres',
        'password': 'admin'
    }
    
    try:
        # Process and save the data locally
        ml_ready_dataset = process_airbnb_data(file_path, save_path)
        
        print("Dataset Shape:", ml_ready_dataset.shape)
        print("\nAvailability Columns:")
        availability_cols = [col for col in ml_ready_dataset.columns if 'availability' in col]
        print(availability_cols)
        
        print("\nAvailability Distribution:")
        print(ml_ready_dataset['availability_category'].value_counts(normalize=True))
        
        print("\nCorrelation between Availability and Price:")
        print(ml_ready_dataset[['availability_365', 'price']].corr())
        
        # Save processed data to PostgreSQL
        save_to_postgres(ml_ready_dataset, 'airbnb_data', db_config)
        
    except Exception as e:
        logging.error(f'An error occurred during processing: {str(e)}')
